# climatecasechart, run surya ocr

In [8]:
#@title Import

import re
import os
import json
import time
import pickle
import requests
import numpy as np
import pandas as pd
from tqdm.cli import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool

from IPython.display import display
from shapely.geometry import box

import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from boxly.ops import iou

tqdm.pandas()


# os.chdir('/slow-data/unitednationsclimatehealth/')
os.chdir('/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/')

base_url = 'https://climatecasechart.com/non-us-jurisdiction/' #@param {"type": "string"}
experiment_date = '2025-03-28-run-01' #@param {"type": "string"}

In [ ]:
#@title Run surya (looooong)

case_details = pd.read_pickle(f'data/{experiment_date}-case-details.p3')

left = case_details[
    # not already finished
    ~case_details['slug'].apply(
        lambda x: os.path.exists(f'data/{experiment_date}-surya-ocr/{x}/results.json'))
    # and actually has documents:
    # & (case_details['documents'].apply(lambda x: [x for x in x if x['File'] != '']).str.len() > 0)
]

def run_ocr(pdf_path, ocr_path):
  print()
  os.makedirs(ocr_path, exist_ok=True)
  if not os.path.exists(f'{ocr_path}/results.json'):
    !CUDA_VISIBLE_DEVICES=2 RECOGNITION_BATCH_SIZE=8 DETECTOR_BATCH_SIZE=64  surya_ocr '{pdf_path}' --output_dir '{ocr_path}'
  print()

for (slug, link), documents in tqdm(left.groupby(['slug', 'link'])['documents'], position=0):
  pdf_path = f'data/{experiment_date}-pdf/{slug}/'
  ocr_path = f'data/{experiment_date}-surya-ocr/{slug}/'
  print()
  print(slug)

  run_ocr(pdf_path, ocr_path)

In [9]:
#@title Re-index ocr dictionaries to actual filename

for slug in tqdm(case_details['slug'].unique()):
  if not os.path.exists(f'data/{experiment_date}-surya-ocr/{slug}/results.json'):
    continue
  # if os.path.exists(f'data/{experiment_date}-surya-ocr/{slug}/fixed-results.json'):
  #   continue
  with open(f'data/{experiment_date}-surya-ocr/{slug}/results.json') as f:
    ocr = json.load(f)
  fns = [fn for fn in os.listdir(f'data/{experiment_date}-pdf/{slug}/') if fn.endswith('.pdf')]
  if len(ocr) == len(fns):
    ocr = {
        min((fn for fn in fns if fn.startswith(k)), key=len): v for k, v in ocr.items()
    }
    with open(f'data/{experiment_date}-surya-ocr/{slug}/fixed-results.json', 'w') as f:
      json.dump(ocr, f)
  else:
    print(slug)

100%|██████████| 332/332 [00:38<00:00,  8.62it/s]
